In [1]:
%pylab
%matplotlib inline


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pyfits as fits
from numpy import polyfit
from astropy.utils.data import download_file
import pandas as pd
from cosmoslik import get_plugin, param_shortcut, SlikPlugin, SlikDict, run_chain

In [3]:
### Download data


### Cepheid table
cepheid_table = download_file(
        'http://iopscience.iop.org/0004-637X/730/2/119/suppdata/apj383673t2_mrt.txt', 
        cache = True)
cepheids = pd.read_csv(cepheid_table,
                       delim_whitespace = True,
                       skiprows = 39,
                       names = (['Field', 'RAdeg', 
                                 'DEdeg', 'ID', 
                                 'Period', 'VtoI', 
                                 'F160Wmag', 'e_F160Wmag',
                                 'Offset', 'Bias', 
                                 'IMrms', 'ObyH', 'Flag']
                               )
                      )

cepheids=cepheids.fillna(value = '-')
### SNe table
Sne_table = download_file(
        'http://iopscience.iop.org/0004-637X/730/2/119/suppdata/apj383673t3_ascii.txt',
        cache = True)
Sne = pd.read_csv(Sne_table, 
                   
                  delim_whitespace=True, 
                  skiprows = [0,1,2,3,4,13,15],
                  names = (['Host', 'junk','Sn1A',
                            'Filters', 'm0_viPlus5a_v',
                            'sigma', 'DeltaMu_0','e_DeltaMu_0',
                            'mu_0_Best', 'e_mu_0_Best'
                          ])
                 )
Sne.loc[:,'e_DeltaMu_0'] = (Sne.loc[:,'e_DeltaMu_0'].apply(str).str.replace('\(|\)','')).astype('float')
Sne.loc[:,'e_mu_0_Best'] = (Sne.loc[:,'e_mu_0_Best'].apply(str).str.replace('\(|\)','')).astype('float')


maser_distance = {'mu':7.2, 'e_mu':0.32} 
maser_distance = pd.DataFrame(data = maser_distance, index = arange(1))







In [7]:
param = param_shortcut('start','scale')
class CosmoModel():
    def __init__(cepheids, sne):
        self.cepheids = cepheids
        self.local_sne = sne
        self.fields = set(self.cepheids.Field)
        
        ####Params####
        ##############
        self.params = SlikDict()
        ### Cepheid period/luminosity parameters
        self.params.b = param(-3, .05)
        self.param.Z = param(-0.27, 0.1)
        self.param.intercept = param(10,.5)

    
        ### Differential distance moduli parameters
        for field in self.fields:
            self.params['dmu_field'] = param(1, 0.1)
        
        ### Cepheid dispersion parameters
        self.params.R = 0.410 ## For calculating Weisenheit reddening-free magnitudes
        
        ### Sne mag/redshift relation
        self.params.abs_mag = param(-20, 0.1)
        
        ### Cosmology parameters
        self.params.H0 = param(70, 1)
        self.params.OmM = param(.2, .01)
        #self.params.OmR = param(0.004, 0.001)
        self.params.OmK = 0

        ### Sne Dispersion Parameters
        
        
        ### Geometric Distance Parameters
        self.params.mu_4258 = param(7.2, .1)
        
        
    def _ceph_period_to_relative_magnitude(self, period, host, deltaOH = None, VminusI = None):
        period = array([period]).flatten()
        host = array([host]).flatten()
        mag = self.params.b * period + self.params.intercept
        if deltaOH != None:
            mag += self.params.Z * deltaOH ### Metallicity axis
        if VminusI != None:
            mag += self.params.R * VminusI
        for field in self.fields:
            mag += self.params.mu_4258 + array(map(lambda y: y == field, host)) * self.params['dmu_'+field]
        return mag
    
    
        
    def SN_z_to_relative_mag(self, z):
        d_lum = self._get_distances(z)['luminosity']
        mag = self.params.abs_mag + 5 * log10(d_lum) + 25
        return mag
    
    def _get_distances(self, z):
        z = array([z]).sort().flatten()
        distances = SlikDict()
        distances['z'] = z
        distances['hubble'] = array([1.0e5/self.params.H0] * z.size())
        zpairs = zip(z[:-1], roll(z, -1)[:-1])
        distances['comoving'] = cumsum(map(self._differential_comoving_distance, zpairs))
        if self.params.OmK == 0:
            distances['chi'] = distances['comoving']
        elif self.params.OmK > 0:
            distances['chi'] = distances['hubble']/sqrt(self.params.OmK) * sinh(sqrt(self.params.OmK)*
                                                                     distances['comoving']/distances['hubble'])
        elif self.params.OmK < 0:
            distances['chi'] = distances['hubble']/sqrt(self.params.OmK) * sin(sqrt(self.params.OmK)*
                                                                    distances['comoving']/distances['hubble'])
        else:
            return -1
        distances['luminosity'] = (1 + z) * distances['chi']
        distances['angular_diameter'] = distances['chi']/(1+z)
        return distances
        
        
    def _differential_comoving_distance(self, zpairs):
        return 1/H_0 * quad(lambda x: 1.0/_E(x), zpairs[0], zpairs[1])
        
    def _E(self, z):
        return sqrt(self.params.OmM*( 1 + z)**3 + self.params.OmK*(1+z)**2 + self.params.OmL)
        
    def __call__(self):
        self.params.OmL = 1 - self.params.OmM - self.params.OmK
        lnl = 0
        return lnl
        
        